IMPORT

In [ ]:
import pandas as pd
import numpy as np
import os
import Levenshtein as lev
from rapidfuzz import fuzz
import re
from itertools import permutations

FUNCTIONS

In [ ]:
def pulisci_stringhe_fuzz(df_temp, colonna, soglia_fuzz):
    df_temp2=df_temp.copy() #poi usare df_temp2
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()


    prima= set(df_temp2[colonna_mod].unique())

    diz = df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1 != val2:
                similarità = fuzz.ratio(val1, val2)
                if similarità >= soglia_fuzz and diz[val1] >= diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1  #Sostituisci tutti i valori uguali a val2 nella colonna colonna_mod del DataFrame df_temp2 con val1.# creare nuova colonna invece di replace
                elif similarità >=soglia_fuzz and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

   
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

     

    return df_temp2, varianti_dict 

In [ ]:
def pulisci_stringhe_fuzz2(df_temp, colonna, soglia_fuzz,colonna_da_uguagliare):
    df_temp2=df_temp.copy() #poi usare df_temp2
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()


    prima= set(df_temp2[colonna_mod].unique())
    diz = df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1 != val2:
                similarità = fuzz.ratio(val1, val2)
                if similarità >= soglia_fuzz and diz[val1] >= diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1  #Sostituisci tutti i valori uguali a val2 nella colonna colonna_mod del DataFrame df_temp2 con val1.# creare nuova colonna invece di replace
                elif similarità >=soglia_fuzz and diz[val1]<diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

   
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}
     

    return df_temp2, varianti_dict

In [ ]:
def pulisci_stringhe_lev(df_temp,colonna,soglia_lev):
    df_temp2=df_temp.copy()
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()
    

    prima= set(df_temp2[colonna_mod].unique())
    diz= df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1!=val2:
                distanza=lev.distance(val1,val2)
                if distanza <= soglia_lev and diz[val1]>=diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1
                elif distanza <= soglia_lev and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

    
    
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}


    return df_temp2, dict

In [ ]:
def pulisci_stringhe_lev2(df_temp,colonna,soglia_lev,colonna_da_uguagliare):
    df_temp2=df_temp.copy()
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()
    

    prima= set(df_temp2[colonna_mod].unique())
    diz= df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1!=val2:
                distanza=lev.distance(val1,val2)
                if distanza <= soglia_lev and diz[val1]>=diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1 
                elif distanza <= soglia_lev and diz[val1]<diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

    
    
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

    return df_temp2, dict

In [ ]:
def pulisci_trattino(val):
    val = str(val)
    # Se c'è un "-" in mezzo, lo sostituisce con un "."
    val = re.sub(r'(?<!^)-', '.', val) #(?<!^) è una negative lookbehind assertion: controlla che prima del trattino non ci sia l'inizio della stringa (^)
    return val

In [ ]:
def creo_dict(df,colonna_originale,colonna_modificata):
    dizionario={key: list(group[colonna_originale].unique())
    for key, group in df.groupby(colonna_modificata)}
    return dizionario

In [ ]:
def creo_dict2(df, colonna_originale, colonna_modificata):
    dizionario = {
        round(key, 2): list(set(
            v if isinstance(v, (int, float)) else v
            for v in group[colonna_originale]
        ))
        for key, group in df.groupby(colonna_modificata)
    }
    return dizionario

In [ ]:
def uniforma_date_order_date(df, colonna_originale1, colonna_originale2):
    #creo una colonna in formato datetime e una in formato stringa
    #quella in formato datetime mi serve per pulire il grosso degli errori
    #quella in formato stringa mi serve per pulire gli errori come per esempio se il mese supera il 12, o se è 0 ecc
    #quindi quando la colonna dt mi da errore, lavoro sulla colonna str
    # scelgo come formato unico il formato YYYY/MM/DD
    colonna1_dt=colonna_originale1 + '_dt'
    colonna2_dt=colonna_originale2 + '_dt'
    colonna1_def=colonna_originale1 + '_def'


    df[colonna1_dt] = pd.to_datetime(df[colonna_originale1], errors='coerce')
    df[colonna2_dt] = pd.to_datetime(df[colonna_originale2], errors='coerce')
    

    #Inizializzo colonna finale
    df[colonna1_def] = df[colonna1_dt]  # dove non è NaT, la data è già buona

    
    for i, row in df[pd.isna(df[colonna1_dt])].iterrows():
        raw_date = str(row[colonna_originale1]) #rendo la data in stringa così posso lavorarci
        try:
            parts = re.split(r'\D+', raw_date.strip()) #cosi faccio lo split non solo sullo / ma qualsiasi simbolo non numerico
            combinazioni = ['/'.join(p) for p in permutations(parts)] #trovo tutte le combinazioni possibili
            date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
            date_valide = date_possibili[date_possibili.notna() & (date_possibili <= row[colonna2_dt])] #controllo che la data possibile sia precedente alla data di spedizione
            if not date_valide.empty:
                distanze = row[colonna2_dt] - date_valide  # sarà sempre >= 0
                indice_minimo = distanze.argmin()
                data_probabile = date_valide.iloc[indice_minimo] #indice della differenza minima
                df.at[i, colonna1_def] = data_probabile
        except:
            print(f'La riga {i} da errore')
    return df

In [ ]:
def uniforma_date_ship_date(df, colonna_originale1, colonna_originale2):
    #creo una colonna in formato datetime e una in formato stringa
    #quella in formato datetime mi serve per pulire il grosso degli errori
    #quella in formato stringa mi serve per pulire gli errori come per esempio se il mese supera il 12, o se è 0 ecc
    #quindi quando la colonna dt mi da errore, lavoro sulla colonna str
    # scelgo come formato unico il formato YYYY/MM/DD
    colonna1_dt=colonna_originale1 + '_dt'
    colonna2_dt=colonna_originale2 + '_dt'
    colonna1_def=colonna_originale1 + '_def'


    df[colonna1_dt] = pd.to_datetime(df[colonna_originale1], errors='coerce')
    df[colonna2_dt] = pd.to_datetime(df[colonna_originale2], errors='coerce')
    

    #Inizializzo colonna finale
    df[colonna1_def] = df[colonna1_dt]  # dove non è NaT, la data è già buona

    
    for i, row in df[pd.isna(df[colonna1_dt])].iterrows():
        raw_date = str(row[colonna_originale1]) #rendo la data in stringa così posso lavorarci
        try:
            parts = re.split(r'\D+', raw_date.strip()) #cosi faccio lo split non solo sullo / ma qualsiasi simbolo non numerico
            combinazioni = ['/'.join(p) for p in permutations(parts)] #trovo tutte le combinazioni possibili
            date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
            date_valide = date_possibili[date_possibili.notna() & (date_possibili >= row[colonna2_dt])] #controllo che la data possibile sia successiva alla data di ordine
            if not date_valide.empty:
                distanze = date_valide - row[colonna2_dt]  # sarà sempre >= 0
                indice_minimo = distanze.argmin()
                data_probabile = date_valide.iloc[indice_minimo] #indice della differenza minima
                df.at[i, colonna1_def] = data_probabile
        except:
            print(f'La riga {i} da errore')
    return df

GET DATA

In [ ]:
# Leggi tutti i fogli come dizionario di DataFrame
xls = pd.ExcelFile("../data/Superstore_mod_v1.xlsx")
df_dict = pd.read_excel(xls, sheet_name=None)  # None legge tutti i fogli

# df_dict è un dizionario con chiavi = nomi dei fogli, valori = DataFrame
for sheet_name, df in df_dict.items():
    print(f"Foglio: {sheet_name}")  
    globals()[f'df_{sheet_name.lower()}'] = df  # globals() in Python serve a restituire un dizionario contenente tutte le variabili globali 
                                                #attualmente disponibili nel contesto in cui viene chiamato

display(df_countries)

FOGLIO COUNTRIES

In [ ]:
'''COLONNA STATE'''
df_countries_raw=df_countries.drop_duplicates() #tolgo i duplicati da tutto il df_countries

#FUNZIONE FUZZ SOGLIA 90
df_countries_fuzz_90, dict_countries_fuzz_90= pulisci_stringhe_fuzz(df_countries_raw,'State',90) 
df_countries_fuzz_90 = df_countries_fuzz_90.rename(columns={'State_mod': 'State_fuzz_90'})
df_countries_fuzz_90['State_fuzz_90'] = df_countries_fuzz_90['State_fuzz_90'].replace({
    'NY': 'NEW YORK', 
    'NC': 'NORTH CAROLINA'
})
df_stati_corretti_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_90.items() if len(v)>1], columns=['State', 'Variants_fuzz_90'])

#FUNZIONE FUZZ SOGLIA 80
df_countries_fuzz_80,dict_countries_fuzz_80= pulisci_stringhe_fuzz(df_countries_fuzz_90,'State',80)
df_countries_fuzz_80= df_countries_fuzz_80.rename(columns={'State_mod':'State_fuzz_80'})
df_countries_fuzz_80['State_fuzz_80']=df_countries_fuzz_80['State_fuzz_80'].replace({'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA'}) 
df_stati_corretti_fuzz_80=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_80.items() if len(v)>1], columns=['State', 'Variants_fuzz_80'])


#FUNZIONE LEV SOGLIA 1
df_countries_lev_1, dict_countries_lev_1=pulisci_stringhe_lev(df_countries_fuzz_80,'State',1) #applico la funzione lev
df_countries_lev_1= df_countries_lev_1.rename(columns={'State_mod':'State_lev_1'})
df_countries_lev_1['State_lev_1']=df_countries_lev_1['State_lev_1'].replace({'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA'}) 
#df_countries_lev_1, guardando gli output, è quello che si avvicina di più ad essere corretto
state_map = {
    'NC': 'NORTH CAROLINA'
}
df_countries_lev_1['State_lev_1'] = df_countries_lev_1['State'].map(state_map).fillna(df_countries_lev_1['State_lev_1'])
df_stati_corretti_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_1.items() if len(v) >1], columns=['State', 'Variants_lev_1'])

#MERGE TRA I 3 DF
df_merge_state_lev_fuzz90 = df_stati_corretti_fuzz_90.merge(df_stati_corretti_lev_1, on = 'State', how = 'outer')
df_merge_state_lev_fuzz90_fuzz80= df_merge_state_lev_fuzz90.merge(df_stati_corretti_fuzz_80, on='State', how='outer' )

'''LI CARICO SU EXCEL
df_merge_state_lev_fuzz90_fuzz80.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Countries_up.xlsx', sheet_name='Country', index=False)
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Countries_up.xlsx', engine='openpyxl', mode='a') as writer:
    df_merge_state_lev_fuzz90_fuzz80.to_excel(writer, sheet_name='Country(no len)', index=False) '''
    

In [ ]:
'''COLONNA CITY'''

#FUNZIONE FUZZ CON SOGLIA 90
df_countries_fuzz_city_90, dict_countries_fuzz_city_90= pulisci_stringhe_fuzz(df_countries_lev_1,'City',90) 
df_countries_fuzz_city_90 = df_countries_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_countries_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_90.items() ], columns=['City', 'Variants_fuzz_90'])


#FUNZIONE FUZZ CON SOGLIA 95
df_countries_fuzz_city_95,dict_countries_fuzz_city_95= pulisci_stringhe_fuzz(df_countries_fuzz_city_90,'City',95) 
df_countries_fuzz_city_95 = df_countries_fuzz_city_95.rename(columns={'City_mod': 'City_fuzz_95'})
df_countries_fuzz_city_95['City_fuzz_95']= df_countries_fuzz_city_95['City_fuzz_95'].replace({'BOWLINGGREEN': 'BOWLING GREEN'}).replace({'WOODSTOK': 'WOODSTOCK'})
df_countries_city_corrette_fuzz_95=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_95.items() ], columns=['City', 'Variants_fuzz_95'])
df_countries_fuzz_city_95 = df_countries_fuzz_city_95.rename(columns={'City_fuzz_95': 'City_def'})
df_countries_fuzz_city_95['City_def']=df_countries_fuzz_city_95['City_def'].replace({
    'ENDERSON':'HENDERSON'
})



#FUNZIONE LEV CON SOGLIA 1
df_countries_lev_city_1, dict_countries_lev_city_1= pulisci_stringhe_lev(df_countries_fuzz_city_95,'City',1) 
df_countries_lev_city_1 = df_countries_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
df_countries_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_city_1.items()], columns=['City', 'Variants_lev_1'])


#MERGE TRA I 3 DF
df_merge_city_fuzz_95_fuzz_90=df_countries_city_corrette_fuzz_95.merge(df_countries_city_corrette_fuzz_90, on='City', how='outer')
df_merge_city_fuzz_95_fuzz_90_lev_1=df_countries_city_corrette_lev_1.merge(df_merge_city_fuzz_95_fuzz_90, on='City', how='outer')


'''LI CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Countries_up.xlsx', engine='openpyxl', mode='a') as writer:
    df_merge_city_fuzz_95_fuzz_90_lev_1.to_excel(writer, sheet_name='City(no len)', index=False)'''

#METODO PIù GIUSTO: FUZZ CON SOGLIA=95 ci sono pochi errori in questa colonna city, ce ne sono molti nella colonna city del foglio orders, ancora da finire


df_countries_lev_city_1=df_countries_lev_city_1[['Country','City','City_fuzz_90','City_def','City_lev_1','State','State_fuzz_90','State_fuzz_80','State_lev_1','Postal Code','Region']]

In [ ]:
''' ANCORA COLONNA CITY MA CON CONTROLLO DELLO STATO PER MODIFICARE LA CITTA: MODIFICO LA CITTA' SOLO SE HANNO LO STATO UGUALE'''

#METODO FUZZ (2) CON SOGLIA 90
df_countries_fuzz_city_90_c, dict_countries_fuzz_city_90_c= pulisci_stringhe_fuzz2(df_countries_lev_1,'City',90,'State_lev_1') 
df_countries_fuzz_city_90_c = df_countries_fuzz_city_90_c.rename(columns={'City_mod': 'City_fuzz_90'})
df_countries_city_corrette_fuzz_90_c=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_90_c.items() ], columns=['City', 'Variants_fuzz_90'])


#METODO FUZZ (2) CON SOGLIA 95
df_countries_fuzz_city_95_c,dict_countries_fuzz_city_95_c= pulisci_stringhe_fuzz2(df_countries_fuzz_city_90_c,'City',95,'State_lev_1') 
df_countries_fuzz_city_95_c = df_countries_fuzz_city_95_c.rename(columns={'City_mod': 'City_fuzz_95'})
df_countries_city_corrette_fuzz_95_c=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_95_c.items() ], columns=['City', 'Variants_fuzz_95'])



#METODO LEV (2) CON SOGLIA 1
df_countries_lev_city_1_c, dict_countries_lev_city_1_c= pulisci_stringhe_lev2(df_countries_fuzz_city_95_c,'City',1,'State_lev_1') 
df_countries_lev_city_1_c = df_countries_lev_city_1_c.rename(columns={'City_mod': 'City_lev_1'})
df_countries_city_corrette_lev_1_c=pd.DataFrame([(k, v) for k, v in dict_countries_lev_city_1_c.items()], columns=['City', 'Variants_lev_1'])


#MERGE TRA I 3 DF
df_merge_city_fuzz_95_fuzz_90_c=df_countries_city_corrette_fuzz_95_c.merge(df_countries_city_corrette_fuzz_90_c, on='City', how='outer')
df_merge_city_fuzz_95_fuzz_90_lev_1_c=df_countries_city_corrette_lev_1_c.merge(df_merge_city_fuzz_95_fuzz_90_c, on='City', how='outer')


'''LI CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Countries_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_merge_city_fuzz_95_fuzz_90_lev_1_c.to_excel(writer, sheet_name='City_c', index=False)'''

FOGLIO ORDERS

In [ ]:
df_orders_raw=df_orders
df_orders_mod=df_orders_raw.copy().drop_duplicates()

'''COLONNA SHIP MODE'''  
#METODO LEV CON SOGLIA 5
df_orders_ship_mode_lev_1, dict_orders_ship_mode_lev_1 =pulisci_stringhe_lev(df_orders_mod,'Ship Mode',5)


'''LO CARICO SU EXCEL
df_orders_ship_mode_corrette=pd.DataFrame([(k, v) for k, v in dict_orders_ship_mode_lev_1.items() if len(v)>1], columns=['Ship Mode', 'Variants_lev_1'])
df_orders_ship_mode_corrette.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', sheet_name='Ship Mode', index=False)'''


In [ ]:
'''COLONNA SALES'''

df_orders_ship_mode_lev_1['Sales_mod']= df_orders_ship_mode_lev_1['Sales'].apply(pulisci_trattino) # sostitiusco il - con il . quando è in mezzo alla stringa
df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].astype(str).str.strip() # rimuovo eventuali spazi bianchi e converto in string
df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # sostituisco le virgole con punti (per uniformare la notazione decimale), il ' e la a

df_orders_ship_mode_lev_1['Sales_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Sales_mod'], errors='coerce')
dict_sales_mod = creo_dict2(df_orders_ship_mode_lev_1,'Sales','Sales_mod')
df_sales_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_sales_mod.items()], columns=['Sales', 'Variants'])

'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
    df_sales_mod_corrette.to_excel(writer, sheet_name='Sales.round2)', index=False)'''


df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Ship Date','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Discount','Profit']] #avvicino Sales_mod a Sales

In [ ]:
'''COLONNA QUANTITY'''

df_orders_ship_mode_lev_1['Quantity_mod']=df_orders_ship_mode_lev_1['Quantity'].astype(str).apply(pulisci_trattino)
df_orders_ship_mode_lev_1['Quantity_mod'] = df_orders_ship_mode_lev_1['Quantity_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('ì', '', regex=False) # sostituisco le virgole con punti (per uniformare la notazione decimale), il ' e il ì

df_orders_ship_mode_lev_1['Quantity_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Quantity_mod'], errors='coerce')
dict_quantity_mod=creo_dict2(df_orders_ship_mode_lev_1,'Quantity','Quantity_mod')
df_quantity_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_quantity_mod.items() if len(v)>1], columns=['Quantity', 'Variants'])

'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_quantity_mod_corrette.to_excel(writer, sheet_name='Quantity.round', index=False)'''

In [ ]:
'''COLONNA DISCOUNT'''

df_orders_ship_mode_lev_1['Discount_mod'] = df_orders_ship_mode_lev_1['Discount'].astype(str).str.replace("ù", '', regex=False).str.replace("%", '', regex=False)

df_orders_ship_mode_lev_1['Discount_mod']=pd.to_numeric(df_orders_ship_mode_lev_1['Discount_mod'],errors='coerce')
dict_discount_mod=creo_dict(df_orders_ship_mode_lev_1,'Discount','Discount_mod')
df_discount_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_discount_mod.items() if len(v)>1], columns=['Discount', 'Variants'])

'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_discount_mod_corretto.to_excel(writer, sheet_name='Discount', index=False)'''

In [ ]:
'''COLONNA PROFIT'''

df_orders_ship_mode_lev_1['Profit_mod']= df_orders_ship_mode_lev_1['Profit'].astype(str).apply(pulisci_trattino)
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.strip() # 1. Rimuovi eventuali spazi bianchi e converti in string
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # 2. Sostituisci le virgole con punti (per uniformare la notazione decimale)

df_orders_ship_mode_lev_1['Profit_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Profit_mod'], errors='coerce')
dict_profit_mod=creo_dict2(df_orders_ship_mode_lev_1,'Profit','Profit_mod')
df_profit_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_profit_mod.items() if len(v)>1 ], columns=['Profit', 'Variants'])

'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
  df_profit_mod_corretto.to_excel(writer, sheet_name='Profit.round2', index=False)'''

In [ ]:
'''COLONNA CITY'''

#FUNZIONE FUZZ CON SOGLIA 90
df_orders_fuzz_city_90, dict_orders_city_fuzz_90= pulisci_stringhe_fuzz(df_orders_ship_mode_lev_1,'City',90) 
df_orders_fuzz_city_90 = df_orders_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_orders_fuzz_city_90['City_fuzz_90']= df_orders_fuzz_city_90['City_fuzz_90'].replace({'BOWLINGGREEN': 'BOWLING GREEN'})
df_orders_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_90.items() if len(v)>1], columns=['City', 'Variants_fuzz_90'])

#FUNZIONE FUZZ CON SOGLIA 85
df_orders_fuzz_city_85,dict_orders_city_fuzz_85=pulisci_stringhe_fuzz(df_orders_fuzz_city_90,'City',85) 
df_orders_fuzz_city_85 = df_orders_fuzz_city_85.rename(columns={'City_mod': 'City_fuzz_85'})
df_orders_city_corrette_fuzz_85=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_85.items() if len(v)>1], columns=['City', 'Variants_fuzz_85'])



#FUNZIONE LEV CON SOGLIA 1
df_orders_lev_city_1,dict_orders_city_lev_1=pulisci_stringhe_lev(df_orders_fuzz_city_85,'City',1) # quello con meno errori
df_orders_lev_city_1 = df_orders_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
citta_da_ripristinare = ['Conroe', 'Redding', 'Camarillo', 'Englewood', 'Mason', 'Medford', 'Normal', 'Layton', 'Renton','Clifton','Edmond'] #SISTEMO LE CITTà CHE HA RIMOSSO MA NON DOVEVA
mask = df_orders_lev_city_1['City'].isin(citta_da_ripristinare)
df_orders_lev_city_1.loc[mask, 'City_lev_1'] = df_orders_lev_city_1.loc[mask, 'City'].str.upper()
df_orders_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_1.items() if len(v)>1], columns=['City', 'Variants_lev_1'])
df_orders_lev_city_1['City_lev_1']=df_orders_lev_city_1['City_lev_1'].replace({
    'ALECSANDRIA':'ALEXANDRIA',
    'LA':'LOS ANGELES',
    'MARLBOROUG':'MARLBOROUGH',
    'FILADELPHIA':'PHILADELPHIA',
    'NY CITY':'NEW YORK CITY',
    'NYC':'NEW YORK CITY',
    'EDINBURG':'EDINBURGH'
})
df_orders_lev_city_1=df_orders_lev_city_1.rename(columns={'City_lev_1':'City_def'})


#FUNZIONE LEV CON SOGLIA 2
df_orders_lev_city_2,dict_orders_city_lev_2=pulisci_stringhe_lev(df_orders_lev_city_1,'City',2) #non va bene, rimuove troppe città corrette
df_orders_lev_city_2 = df_orders_lev_city_2.rename(columns={'City_mod': 'City_lev_2'})
df_orders_city_corrette_lev_2=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_2.items() if len(v)>1], columns=['City', 'Variants_lev_2'])



#FACCIO IL MERGE TRA I 4 DF
df_merge_orders_city_fuzz_90_fuzz_85=df_orders_city_corrette_fuzz_90.merge(df_orders_city_corrette_fuzz_85, on='City',how='outer')
df_merge_orders_city_fuzz_90_fuzz_85_lev_1=df_orders_city_corrette_lev_1.merge(df_merge_orders_city_fuzz_90_fuzz_85, on='City', how='outer')
df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2=df_orders_city_corrette_lev_2.merge(df_merge_orders_city_fuzz_90_fuzz_85_lev_1, on='City', how='outer')


'''LI CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2.to_excel(writer, sheet_name='City', index=False)'''


In [ ]:
'''COLONNA ORDER DATE'''
df_date_pulite=uniforma_date_order_date(df_orders_lev_city_1,'Order Date','Ship Date')
dict_order_date=creo_dict(df_date_pulite,'Order Date','Order Date_def')
df_order_date_corrette=pd.DataFrame([(k, v) for k, v in dict_order_date.items()], columns=['Order Date', 'Variants'])

#C'è ANCORA IL PROBLEMA DELLE RIGHE COME RIGA (422 OPPURE 513) CON LA DATA 13/13/2014 CHE ANCHE SE GUARDO LE DIVERSE COMBINAZIONI, NON HA NESSUNA DATA VALIDA, PER QUESTE DUE USO IL REPLACE
df_date_pulite.loc[df_date_pulite['Order Date'] == '13/13/2014', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2014-10-13')
df_date_pulite.loc[df_date_pulite['Order Date'] == '25/13/2011', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2011-11-25')
df_date_pulite.loc[df_date_pulite['Order Date'] == '14/03/013', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2013-03-14')

df_date_pulite=df_date_pulite.drop(columns=['Order Date_dt','Ship Date_dt'])


'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_order_date_corrette.to_excel(writer, sheet_name='Order Date(no len)', index=False)'''

In [ ]:
'''COLONNA SHIP DATE'''
df_date_pulite2=uniforma_date_ship_date(df_date_pulite,'Ship Date', 'Order Date')
dict_ship_date=creo_dict(df_date_pulite2,'Ship Date','Ship Date_def')
df_ship_date_corrette=pd.DataFrame([(k, v) for k, v in dict_ship_date.items() ], columns=['Ship Date', 'Variants'])


#devo pulire ancora le date in cui nessuna combniazione mi da una data possibile oppure altri errori
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '02 Sép 2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-09-02')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '22/0/2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-10-22')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '28/13/2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-05-28')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '15/09/013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-09-15')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '10_12_2012', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2012-12-10')
df_date_pulite2=df_date_pulite2.drop(columns=['Ship Date_dt','Order Date_dt'])


'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/fogli excel up/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_ship_date_corrette.to_excel(writer, sheet_name='Ship Date(no len)', index=False)'''

In [ ]:
'''COLONNA PRODUCT ID'''
df_date_pulite2['Product ID_def']=df_date_pulite2['Product ID'].copy()

df_date_pulite2['Product ID_def'] = (
    df_date_pulite2['Product ID_def']
    .str.replace('ù', '', regex=False)
    .str.upper()
    .str.replace(' ', '', regex=False)
    .str.replace('-', '', regex=False)
)


#Applico il formato AAA-BB-1234567
df_date_pulite2['Product ID_def'] = (
    df_date_pulite2['Product ID_def'].str[:3] + '-' +
    df_date_pulite2['Product ID_def'].str[3:5] + '-' +
    df_date_pulite2['Product ID_def'].str[5:]
)


FOGLIO PRODUCTS

In [ ]:
df_products_raw = df_products
df_products_mod=df_products_raw.copy().drop_duplicates()


'''COLONNA CATEGORY'''
#METODO LEV CON SOGLIA 6
df_products_cat_lev_6, dict_products_cat=pulisci_stringhe_lev(df_products_mod,'Category',6)
df_products_cat_corrette=pd.DataFrame([(k, v) for k, v in dict_products_cat.items()  if len(v)>1], columns=['Category', 'Variants'])

'''LO CARICO SU EXCEL
df_products_cat_corrette.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Products_up.xlsx', sheet_name='Category', index=False)
'''

In [ ]:
'''COLONNA SUB-CATEGORY'''
#METODO LEV CON SOGLIA 2
df_products_sub_lev2, dict_products_sub=pulisci_stringhe_lev(df_products_cat_lev_6,'Sub-Category',2)
df_products_sub_corrette=pd.DataFrame([(k, v) for k, v in dict_products_sub.items() if len(v)>1], columns=['Sub-Category', 'Variants'])

'''LO CARICO SU EXCEL
#with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Products_up.xlsx', engine='openpyxl', mode='a') as writer:
#   df_products_sub_corrette.to_excel(writer, sheet_name='Sub-Category', index=False)'''

df_products_sub_lev2=df_products_sub_lev2[['Product ID','Category','Category_mod','Sub-Category','Sub-Category_mod','Product Name']]#cambio ordine delle colonnne

In [ ]:
'''COLONNA PRODUCT-ID'''
df_products_sub_lev2['Product ID_mod']=df_products_sub_lev2['Product ID']

for index, row in df_products_sub_lev2.iterrows():
    product_id = row['Product ID_mod']
    category = row['Category_mod']
    sub_category= row['Sub-Category_mod']

    if pd.notnull(product_id) and '-' in product_id:
        product_cat = product_id.split('-')[0]
        product_sub_cat = product_id.split('-')[1]

        if product_cat != category[:3]:  # primi 3 caratteri della stringa
            print(f'La riga {index} è sbagliata: ID = {product_id}, Categoria = {category}')
        
        elif product_sub_cat != sub_category[:2]:
            print(f'La riga {index} è sbagliata: ID= {product_id}, Sub-Category= {product_sub_cat}')
            #OK NO ERRORI

FOGLIO CUSTOMERS

In [ ]:
df_customers_raw=df_customers
df_customers_mod=df_customers_raw.copy().drop_duplicates()

'''COLONNA SEGMENT'''
#METODO LEV CON SOGLIA 2
df_customers_lev_2, dict_segment=pulisci_stringhe_lev(df_customers_mod,'Segment',2)
df_customers_segment_corretto=pd.DataFrame([(k, v) for k, v in dict_segment.items() if len(v)>1], columns=['Segment', 'Variants'])

'''LO CARICO SU EXCEL
df_customers_segment_corretto.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Customers_up.xlsx', sheet_name='Segment', index=False)
'''

In [ ]:
df_customers_lev_2['Customer ID_mod']=df_customers_lev_2['Customer ID'].copy()
df_customers_lev_2['Customer Name_mod'] = (df_customers_lev_2['Customer Name'].str.replace('-', ' ', regex=False).str.replace('_', ' ', regex=False))

'''COLONNA CUSTOMER ID'''
for index in df_customers_lev_2.index:
    customer_id = df_customers_lev_2.at[index, 'Customer ID_mod']
    customer_full_name = df_customers_lev_2.at[index, 'Customer Name_mod']
    
    if pd.notnull(customer_id) and '-' in customer_id:
        customer_id_first_letter_name = customer_id[0]
        customer_id_first_letter_surname = customer_id[1]

        parts = customer_full_name.strip().split()
        if len(parts) >= 2:
            first_letter_name = parts[0][0]
            first_letter_surname = parts[1][0]
        else:
            # Gestione nome-cognome attaccati tipo 'CoreyLock'
            split_parts = re.split(r'(?=[A-Z])', customer_full_name)
            if len(split_parts) >= 3:
                first_letter_name = split_parts[1][0]
                first_letter_surname = split_parts[2][0]
            else:
                continue  # nome non gestibile, salta

        if (
            customer_id_first_letter_name != first_letter_name or
            customer_id_first_letter_surname != first_letter_surname
        ):
            print(f'La riga {index} è errata: il Customer ID è {customer_id}, il Customer Name è {customer_full_name}')
            df_customers_lev_2.at[index, 'Customer ID_mod'] = first_letter_name + first_letter_surname + customer_id[2:]

            
dict_customers_id_corretti=creo_dict(df_customers_lev_2,'Customer ID','Customer ID_mod')
df_customers_id_corretti=pd.DataFrame([(k, v) for k, v in dict_customers_id_corretti.items()], columns=['Customer ID', 'Variants'])

'''LO CARICO SU EXCEL
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Customers_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_customers_id_corretti.to_excel(writer, sheet_name='Customer ID', index=False)'''


MERGE TRA ORDERS E COUNTRIES SULLE CITTA'

In [ ]:
df_date_pulite2['Postal Code_mod'] = df_date_pulite2['Postal Code'].astype(str)
df_countries_fuzz_city_95['Postal Code_mod'] = df_countries_fuzz_city_95['Postal Code'].astype(str)

df_merge = pd.merge(df_date_pulite2.drop(columns=['Order Date','Ship Date','Ship Mode','Postal Code','Sales','Quantity','Discount','Profit','City_fuzz_90','City_fuzz_85','City','Product ID']), df_countries_fuzz_city_95.drop(columns=['City','Postal Code','State_fuzz_90','State_fuzz_80','State','City_fuzz_90']), how='left', on=(['City_def','Postal Code_mod']))
pd.set_option('display.max_columns', None) #visualizzo tutte le colonne dei df


df_merge=df_merge[['Order ID','Product ID_def','Customer ID','Country','Region','State_lev_1','City_def','Postal Code_mod','Order Date_def','Ship Date_def','Ship Mode_mod','Sales_mod','Profit_mod','Discount_mod','Quantity_mod']]
df_merge=df_merge.rename(columns={'State_lev_1':'State_def',
                                  'Customer ID':'Customer ID_def',
                                  'Postal Code_mod':'Postal Code_def',
                                  'Profit_mod':'Profit_def',
                                  'Sales_mod':'Sales_def',
                                  'Ship Mode_mod':'Ship Mode_def',
                                  'Discount_mod':'Discount_def',
                                  'Quantity_mod':'Quantity_def',
                                  })

MERGE TRA IL DF CON GIA ORDERS E COUNTRIES E IL DF CUSTOMERS CON CHIAVE CUSTOMER ID_DEF

In [ ]:
df_customers_lev_2=df_customers_lev_2.rename(columns={'Customer ID_mod':'Customer ID_def'})
df_merge_customers=df_merge.merge(df_customers_lev_2.drop(columns=['Segment','Customer ID','Customer Name']), on='Customer ID_def', how='left')
df_merge_customers=df_merge_customers.rename(columns={'Segment_mod':'Segment_def',
                                                      'Customer Name_mod':'Customer Name_def'})


MERGE TRA DF GIA CON I 3 FOGLI E IL DF PRODUCTS CON CHIAVE PRODUCT ID_DEF

In [ ]:
df_products_sub_lev2=df_products_sub_lev2.rename(columns={'Product ID_mod':'Product ID_def'})

df_merge_def=df_merge_customers.merge(df_products_sub_lev2.drop(columns=['Product Name','Category','Sub-Category','Product ID']).drop_duplicates(), on='Product ID_def',how='left')


'''PULISCO ERRORI SU PRODUCT ID'''
df_merge_def=df_merge_def.rename(columns={'Product ID_def':'Product ID_mod'})
df_merge_def['Product ID']=df_merge_def['Product ID_mod'].copy()


for index, row in df_merge_def.iterrows():
    if pd.isna(row['Category_mod']):
        product_parts=row['Product ID_mod'].split('-')
        categoria=product_parts[0]
        sotto_categoria=product_parts[1]
        codice_numerico=product_parts[2]
        if categoria.startswith('OF') and categoria != 'OFF' and len(categoria) == 3:
            nuova_categoria=categoria[0]+'F'+categoria[1]
            ultima_lettera_categoria=categoria[2]
            
            nuova_sotto_categoria=ultima_lettera_categoria + sotto_categoria[:-1]
            ultima_lettera_sotto_categoria= sotto_categoria[-1]

            nuovo_codice_numerico=ultima_lettera_sotto_categoria + codice_numerico

            nuovo_product_id = f"{nuova_categoria}-{nuova_sotto_categoria}-{nuovo_codice_numerico}"

            # Applica la modifica
            df_merge_def.at[index, 'Product ID_mod'] = nuovo_product_id

#FACCIO IL REPLACE PER ERRORI CHE NON SO COME CORREGGERE
df_merge_def['Product ID_mod']=df_merge_def['Product ID_mod'].replace('FUR-BO-10000468','FUR-BO-1000468').replace('FUR-U1-0001935','FUR-FU-10001935').replace('OFF-BI-1000948','OFF-BI-10000948').replace('OFF-ST-10001128','OFF-ST-10001228').replace('OFF-21-0002049','OFF-BI-10002049').replace('FUR-BO-10002206','FUR-BO-1002206').replace('TEC-PH-O10002555','TEC-PH-10002555').replace('OFF-PA-1001274','OFF-PA-10001274').replace('OFF-PA-1003724','OFF-PA-10003724')

df_merge_def=df_merge_def.rename(columns={'Product ID_mod':'Product ID_def'})
df_merge_def=df_merge_def.drop(columns=['Product ID','Category_mod','Sub-Category_mod']).merge(df_products_sub_lev2.drop(columns=['Product Name','Category','Sub-Category','Product ID']).drop_duplicates(), on='Product ID_def',how='left')
df_merge_def=df_merge_def.rename(columns={'Category_mod':'Category_def', 'Sub-Category_mod':'Sub-Category_def'})



CREO CSV CON IL DF FINALE

In [ ]:
#df_merge_def.to_csv('MasterTable.csv', index=False)